<a href="https://colab.research.google.com/github/steffen74/EALTA-2019-Workshop/blob/master/azubi_evaluation_ml_with_tensorflow/code/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Import

### Local import

In [ ]:
import os
os.chdir("C:/Users/Steffen/Arbeit/99_opencampus/13_ASCOT+/NLP-Paper/Special-Issue-NLP/azubi_evaluation_ml_with_tensorflow/data")

import numpy
import pandas as pd
data = pd.read_csv("data_new.csv", encoding="UTF-8")

### From Google Drive in Colab

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
import os
os.chdir("./gdrive/MyDrive/NLP-Paper/data")

import numpy
import pandas as pd
data = pd.read_csv("data_new.csv", encoding="UTF-8")

## Data Preparation

In [24]:
# split the data into two pieces, one for training and one for testing
from sklearn.model_selection import train_test_split
X_train_series, X_test_series, y_train_series, y_test_series = train_test_split(data["x"], data["y"], test_size = 0.20, random_state = 42)

# split the training data set again to additionally get a validation data set
from sklearn.model_selection import train_test_split
X_train_series, X_valid_series, y_train_series, y_valid_series = train_test_split(X_test_series, y_test_series, test_size = 0.20, random_state = 42)

## Tokenizing for Fine-Tuning

In [ ]:
from transformers import AutoTokenizer
import numpy as np

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

train_emcodings = tokenizer()

def tokenize_dataset(dataset):
    encoded = tokenizer(
        dataset["sentence1"],
        dataset["sentence2"],
        padding=True,
        truncation=True,
        return_tensors='np',
    )
    return encoded.data

tokenized_datasets = {
    split: tokenize_dataset(raw_datasets[split]) for split in raw_datasets.keys()
}

In [ ]:
!pip install transformers[sentencepiece]
!pip install datasets

In [14]:


from transformers import AutoTokenizer
import numpy as np
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_dataset(dataset):
    encoded = tokenizer(
        dataset["sentence1"],
        dataset["sentence2"],
        padding=True,
        truncation=True,
        return_tensors='np',
    )
    return encoded.data

tokenized_datasets = {
    split: tokenize_dataset(raw_datasets[split]) for split in raw_datasets.keys()
}

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

...

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [22]:
raw_datasets['train'][0]


{'idx': 0,
 'label': 1,
 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}

In [23]:
tokenized_datasets

{'test': {'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         ...,
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]]),
  'input_ids': array([[  101,  7473,  2278, ...,     0,     0,     0],
         [  101,  1996,  2088, ...,     0,     0,     0],
         [  101,  2429,  2000, ...,     0,     0,     0],
         ...,
         [  101, 16559,  5226, ...,     0,     0,     0],
         [  101,  2197,  2733, ...,     0,     0,     0],
         [  101, 17540,  8004, ...,     0,     0,     0]]),
  'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]])},
 'train': {'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         

In [16]:
raw_datasets.keys()

dict_keys(['train', 'validation', 'test'])

## Fine-Tuning

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

model.compile(
    optimizer='adam',
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)
model.fit(
    tokenized_datasets['train'],
    np.array(raw_datasets['train']['label']), 
    validation_data=(
        tokenized_datasets['validation'],
        np.array(raw_datasets['validation']['label']),
    ),
    batch_size=8,
    epochs=3
)

## Fine-Tuning with learning rate optimization

In [ ]:
# Definition of the learning rate scheduler 
from tensorflow.keras.optimizers.schedules import PolynomialDecay
batch_size = 8
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs
num_train_steps = (len(tokenized_datasets['train']['input_ids']) // batch_size) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0.,
    decay_steps=num_train_steps
    )
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=lr_scheduler)

# Definition of the reminaing model parts
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
loss = SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

# Estimation of the model
model.fit(
    tokenized_datasets['train'],
    np.array(raw_datasets['train']['label']), 
    validation_data=(
        tokenized_datasets['validation'],
        np.array(raw_datasets['validation']['label']),
    ),
    batch_size=batch_size,
    epochs=num_epochs
)

## Prediction

In [ ]:
preds = model.predict(tokenized_datasets['validation'])['logits']

In [ ]:
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

from datasets import load_metric

metric = load_metric("glue", "mrpc")
metric.compute(predictions=class_preds, references=raw_datasets['validation']['label'])